In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [11]:
# Yemekler ve içeceklerin isimleri
foods = ['Spaghetti', 'Pizza', 'Burger', 'Steak', 'Salad', 'Soup', 'Coffee', 'Tea', 'Cola', 'Lemonade']

# Malzemeler
ingredients = ['Pasta', 'Tomato sauce', 'Mozzarella', 'Beef', 'Bun', 'Lettuce', 'Tomato', 'Carrot', 'Potato', 'Chicken', 'Cream', 'Sugar', 'Water', 'Ice', 'Lemon']

# Yemeklerin kalorileri
calories = [500, 600, 700, 800, 200, 300, 50, 30, 100, 150]

# Yemeklerin Fiyatları
prices = [10, 12, 8, 18, 7, 5, 3, 2, 1, 2]

data = pd.DataFrame(columns=['Food', 'Ingredient1', 'Ingredient2', 'Ingredient3', 'Calories', 'Price', 'Order'])

for i in range(1000):
    food = np.random.choice(foods)
    ingredient1 = np.random.choice(ingredients)
    ingredient2 = np.random.choice(ingredients)
    ingredient3 = np.random.choice(ingredients)
    calories = np.random.randint(100, 1000)
    price = np.random.choice(prices)
    order = np.random.choice([0, 1])
    data1 = {
        'Food': food,
        'Ingredient1': ingredient1,
        'Ingredient2': ingredient2,
        'Ingredient3': ingredient3,
        'Calories': calories,
        'Price': price,
        'Order': order
    }
    data.loc[len(data)] = data1
data.head()


,Food,Ingredient1,Ingredient2,Ingredient3,Calories,Price,Order
0,Coffee,Potato,Potato,Sugar,635,12,0
1,Spaghetti,Bun,Tomato sauce,Beef,976,18,0
2,Cola,Mozzarella,Mozzarella,Pasta,246,2,1
3,Tea,Mozzarella,Chicken,Beef,245,5,0
4,Cola,Bun,Lettuce,Potato,519,5,1


#### Food: Müşterilerin seçebileceği yemek türleri
#### Ingredient1, Ingredient2, Ingredient3: Yemeklerin içinde bulunan malzemeler
#### Calories: Yemeklerin kalori değeri
#### Price: Yemeklerin fiyatı
#### Order: Müşterilerin sipariş verip vermediği bilgisi (0 veya 1)

# Model İşlemleri

In [12]:
# kategorik değişkenleri numerik değişkenlere dönüştürelim.
lbe = LabelEncoder()
data['Food'] = lbe.fit_transform(data['Food'])
data['Ingredient1'] = lbe.fit_transform(data['Ingredient1'])
data['Ingredient2'] = lbe.fit_transform(data['Ingredient2'])
data['Ingredient3'] = lbe.fit_transform(data['Ingredient3'])

In [13]:
# bağımlı ve bağımsız değişkenlerimizi oluşturalım.
X = data.drop('Order', axis=1)
y = data['Order']

In [14]:
# eğitim ve test setlerini ayıralım.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### KNN algoritması kullanarak bir model kurmak için, öncelikle en iyi hiperparametreleri bulmamız gerekiyor.

In [15]:
knn = KNeighborsClassifier()
params = {
    'n_neighbors':[3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'leaf_size': [10, 20, 30, 40, 50]
}
grid_search = GridSearchCV(estimator=knn, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

print('Best Params: ', grid_search.best_params_)
print('Best Score: ', grid_search.best_score_)

Best Params:  {'algorithm': 'ball_tree', 'leaf_size': 40, 'metric': 'chebyshev', 'n_neighbors': 5, 'weights': 'distance'}
Best Score:  0.5428571428571429


### En iyi parametreler ile bir KNN modeli oluşturalım ve test seti üzerinde doğruluk puanını hesaplayalım.

In [16]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='ball_tree', leaf_size=10, metric='manhattan')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Doğruluk Puanı: ', accuracy)
print('R-Kare Değeri: ', r2)

Doğruluk Puanı:  0.5066666666666667
R-Kare Değeri:  -0.9932659932659933


# Sonuç

### Veri setini kendimiz oluşturduğumuz için gerçek hayat senaryosu değildir. Tamamen knn algoritmasını, mantığını ve kullanımını öğrenmek ve pekiştirmek için yapılmıştır. Bu yüzden tahmin işlemi yapmayacağız. R-kare değeri ve doğruluk değeri de tahmin yapılamayacak kadar kötü bir sonuç :)